In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import os
import random

# Normalization values
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long)

# Prepare and load datasets
def prepare_datasets(paths, labels, sample_size=30000, train_size=29000):
    train_data = []
    test_data = []
    for path, label in zip(paths, labels):
        images = [(os.path.join(path, img), label) for img in os.listdir(path) if img.endswith(('.png', '.jpg', '.jpeg'))]
        if len(images) > sample_size:
            images = random.sample(images, sample_size)
        train_data.extend(images[:train_size])
        test_data.extend(images[train_size:])
    return train_data, test_data

# Paths and labels for datasets
dataset_paths = ["C:\\Users\\ysang\\VOC2012\\JPEGImages", "C:\\Users\\ysang\\flickr30k_images", "C:\\Users\\ysang\\imagenet\\imagenet"]
dataset_labels = [0, 1, 2]  

# Prepare data
train_data, test_data = prepare_datasets(dataset_paths, dataset_labels)

# Transformation pipeline
transform = transforms.Compose([
    transforms.Resize((64, 64)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std),
])

# Create datasets using the transformation
train_dataset = CustomDataset(train_data, transform=transform)
test_dataset = CustomDataset(test_data, transform=transform)

# Dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

# Using ResNet-50 model, modifying it to classify three classes
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 3)  # Modify the fully connected layer
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        running_corrects = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        print(f'Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')

# Evaluation function
def evaluate_model(model, test_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    running_loss = 0.0
    running_corrects = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
    total_loss = running_loss / len(test_loader.dataset)
    total_acc = running_corrects.double() / len(test_loader.dataset)
    print(f'Test Loss: {total_loss:.4f}, Test Acc: {total_acc:.4f}')

# Run training and evaluation
train_model(model, train_loader, criterion, optimizer, num_epochs=10)
evaluate_model(model, test_loader, criterion)


C:\Users\ysang\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ysang\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10 - Loss: 0.4799, Acc: 0.7716
Epoch 2/10 - Loss: 0.4228, Acc: 0.7977
Epoch 3/10 - Loss: 0.3972, Acc: 0.8090
Epoch 4/10 - Loss: 0.3804, Acc: 0.8168
Epoch 5/10 - Loss: 0.3677, Acc: 0.8261
Epoch 6/10 - Loss: 0.3538, Acc: 0.8326
Epoch 7/10 - Loss: 0.3426, Acc: 0.8415
Epoch 8/10 - Loss: 0.3174, Acc: 0.8542
Epoch 9/10 - Loss: 0.2952, Acc: 0.8653
Epoch 10/10 - Loss: 0.2639, Acc: 0.8832
Test Loss: 0.4416, Test Acc: 0.8460
